In [78]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import db_password
import psycopg2
import numpy as np
from flask import Flask
import re

In [62]:
matches_df = pd.read_csv("matches.csv")
matches_df

,loser_age,loser_id,loser_name,loser_rank,winner_age,winner_id,winner_name,winner_rank
0,17.859001,200002,Mirjana Lucic,49,19.296372,200001,Martina Hingis,1
1,27.118412,200004,Kerry Anne Guse,133,17.629021,200003,Justine Henin,63
2,31.378508,200005,Jolene Watanabe Giltz,118,26.458590,200006,Karina Habsudova,53
3,22.006845,200007,Silvija Talaja,23,18.970568,200008,Alicia Molik,116
4,24.821355,200010,Rita Grande,60,22.650240,200009,Tamarine Tanasugarn,72
...,...,...,...,...,...,...,...,...
2442,25.982204,200087,Seda Noorlander,96,23.830253,200040,Anna Smashnova,47
2443,26.160164,200695,Tina Krizan,136,19.542779,201084,Nadejda Ostrovskaya,109
2444,20.533881,200052,Olga Barabanschikova,77,19.175907,200011,Katarina Srebotnik,88
2445,22.691307,201809,Hila Rosen,174,16.569473,201377,Yuliya Beygelzimer,573


In [63]:
matches_df[matches_df["winner_name"]=="Serena Williams"].count()

loser_age      37
loser_id       37
loser_name     37
loser_rank     37
winner_age     37
winner_id      37
winner_name    37
winner_rank    37
dtype: int64

In [64]:
l_stops_df = pd.read_csv("CTA_list_of_L_stops.csv")


#look at the values in the map_id column of the l_stops_df
l_stops_df["MAP_ID"]

0      40830
1      40830
2      40120
3      40120
4      41120
       ...  
295    40540
296    41240
297    40540
298    41700
299    40540
Name: MAP_ID, Length: 300, dtype: int64

In [65]:
l_stops_df.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [66]:
#find number of unique data points in each column of a dataframe

l_stops_df.nunique(axis=0, dropna=True)

STOP_ID                     300
DIRECTION_ID                  4
STOP_NAME                   293
STATION_NAME                108
STATION_DESCRIPTIVE_NAME    143
MAP_ID                      143
ADA                           2
RED                           2
BLUE                          2
G                             2
BRN                           2
P                             2
Pexp                          2
Y                             2
Pnk                           2
O                             2
Location                    143
dtype: int64

In [67]:
#map_id_count = l_stops_df pd.unique(MAP_ID)

unique_map_id = l_stops_df["MAP_ID"].unique()

unique_map_id

array([40830, 40120, 41120, 41270, 40130, 40580, 40990, 40240, 40450,
       40680, 41440, 41420, 41200, 41430, 40660, 41060, 40290, 40010,
       41260, 40060, 41320, 40340, 41380, 40440, 41360, 40280, 40780,
       41000, 41250, 40420, 40970, 40480, 40380, 40430, 41160, 41670,
       40720, 40230, 40210, 40590, 40090, 40050, 40690, 40530, 40320,
       40390, 40520, 40870, 41220, 40510, 40490, 40330, 40760, 41130,
       40940, 40750, 40020, 41490, 41170, 40980, 40850, 40900, 40810,
       40300, 40550, 41460, 40070, 40560, 41190, 41280, 41150, 41040,
       41180, 41070, 40250, 41290, 41140, 40600, 40700, 41340, 40160,
       40770, 41050, 41020, 41300, 40170, 40270, 40460, 40930, 41330,
       41500, 41510, 40100, 41660, 40790, 41090, 40650, 40400, 40180,
       41350, 41310, 41030, 40960, 40150, 40920, 40030, 40040, 40470,
       41680, 40610, 41010, 40890, 41400, 40820, 40800, 40080, 40840,
       40360, 40190, 40260, 40880, 40350, 40140, 40730, 41210, 40310,
       40740, 40370,

In [68]:
l_stops_df = pd.read_csv('CTA_list_of_L_stops.csv')
station_bools = l_stops_df[['MAP_ID','ADA','RED','BLUE','G','BRN','P','Pexp','Y','Pnk','O']].groupby('MAP_ID').any()
l_stations_df = l_stops_df[['MAP_ID','STATION_NAME','STATION_DESCRIPTIVE_NAME','Location']] \
    .merge(station_bools, how='left', left_on='MAP_ID', right_index=True).drop_duplicates()

In [69]:
station_bools

,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
MAP_ID,,,,,,,,,,
40010,False,False,True,False,False,False,False,False,False,False
40020,True,False,False,True,False,False,False,False,False,False
40030,True,False,False,True,False,False,False,False,False,False
40040,False,False,False,False,True,False,True,False,True,True
40050,True,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
41660,True,True,False,False,False,False,False,False,False,False
41670,True,False,False,True,False,False,False,False,False,False
41680,True,False,False,False,False,False,False,True,False,False


In [70]:
l_stations_df[['latitude','longitude']] = \
   l_stations_df['Location'].str.replace('\(|\)','',regex=True) \
   .str.split(',', expand=True).apply(pd.to_numeric)
l_stations_df.drop('Location', axis=1, inplace=True)

In [71]:
ridership_df = pd.read_csv('CTA_ridership_daily_totals.csv')
#ridership_df.head()
ridership_df.dtypes

station_id      int64
stationname    object
date           object
daytype        object
rides           int64
dtype: object

In [72]:
l_stations_df.head()
l_stations_df.dtypes

MAP_ID                        int64
STATION_NAME                 object
STATION_DESCRIPTIVE_NAME     object
ADA                            bool
RED                            bool
BLUE                           bool
G                              bool
BRN                            bool
P                              bool
Pexp                           bool
Y                              bool
Pnk                            bool
O                              bool
latitude                    float64
longitude                   float64
dtype: object

In [74]:
ridership_df.count()

station_id     258109
stationname    258109
date           258109
daytype        258109
rides          258109
dtype: int64

In [79]:
#merge dataframes
frames = [l_stations_df, ridership_df]
result_df = pd.concat(frames)
result_df.notnull().sum()

MAP_ID                         143
STATION_NAME                   143
STATION_DESCRIPTIVE_NAME       143
ADA                            143
RED                            143
BLUE                           143
G                              143
BRN                            143
P                              143
Pexp                           143
Y                              143
Pnk                            143
O                              143
latitude                       143
longitude                      143
station_id                  258109
stationname                 258109
date                        258109
daytype                     258109
rides                       258109
dtype: int64

In [82]:
#find # of rides that serve the southside (less than 41.881 latitude)
southside_rides = result_df[(result_df ["latitude"] < 41.881)] #& (result_df['rides'] > 0)]
southside_rides.sum()

MAP_ID                                                              2483730.0
STATION_NAME                18th35th/Archer35th-Bronzeville-IIT43rd51st54t...
STATION_DESCRIPTIVE_NAME    18th (Pink Line)35th/Archer (Orange Line)35th-...
ADA                                                                        46
RED                                                                        13
BLUE                                                                       15
G                                                                          13
BRN                                                                         4
P                                                                           0
Pexp                                                                        4
Y                                                                           0
Pnk                                                                        15
O                                                               

In [75]:
engine = create_engine("sqlite:///Census_Data.sqlite")

In [76]:
Base = automap_base()

In [77]:
Base.prepare(engine, reflect=True)

In [ ]:
dracula_df = pd.read_csv('dracula.txt', sep='\n', header=None)
dracula_df.columns = ['text']

p = 'YOUR REGULAR EXPRESSION HERE'
dracula_df['text'].str.extractall(p, flags=re.I)[0].value_counts()